In [1]:
from datasets import load_dataset

fleurs_langID = load_dataset("google/fleurs", "all", split="test") # to download all data

/tmp2/b09902033/dgslm/dgslm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset fleurs (/home/student/09/b09902033/.cache/huggingface/datasets/google___fleurs/all/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)


In [28]:
from tqdm import tqdm

l = []

for i in tqdm(range(fleurs_langID.num_rows)):
    l.append(fleurs_langID[i]['audio']['array'].shape)

  7%|▋         | 5199/77810 [00:18<03:53, 311.33it/s]Exception ignored from cffi callback <function SoundFile._init_virtual_io.<locals>.vio_read at 0x7ef5d856e430>:
Traceback (most recent call last):
  File "/tmp2/b09902033/dgslm/dgslm/lib/python3.9/site-packages/soundfile.py", line 1244, in vio_read
    try:
KeyboardInterrupt: 
 16%|█▌        | 12507/77810 [00:49<04:18, 252.46it/s]


KeyboardInterrupt: 

In [23]:
import dataclasses, iso639

def get_iso_693_3(lang_id):
    language = iso639.Language.match(lang_id)
    return dataclasses.asdict(language)['part3']

In [43]:
import whisper
import numpy as np

model = whisper.load_model("base")
preds = []
true_label = []

decode = {i: fleurs_langID.features["lang_id"].names[i].split('_')[0] for i in range(102)}
# load audio and pad/trim it to fit 30 seconds

# fleurs_langID.num_rows
for i in tqdm(range(1000)):
    audio = fleurs_langID[i]['audio']['array'].astype(np.float32)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    pred_id = max(probs, key=probs.get)
    preds.append(get_iso_693_3(pred_id))
    true_label_id = fleurs_langID[i]['lang_id']
    true_label.append(get_iso_693_3(decode[true_label_id]))

preds

100%|██████████| 1000/1000 [01:00<00:00, 16.63it/s]


['nld',
 'lat',
 'afr',
 'sqi',
 'nld',
 'afr',
 'afr',
 'deu',
 'nld',
 'deu',
 'deu',
 'afr',
 'nld',
 'afr',
 'cym',
 'nld',
 'deu',
 'lav',
 'afr',
 'afr',
 'nld',
 'nno',
 'afr',
 'nld',
 'yid',
 'afr',
 'sqi',
 'nld',
 'nld',
 'nno',
 'nld',
 'nno',
 'afr',
 'nld',
 'nld',
 'afr',
 'deu',
 'eng',
 'afr',
 'afr',
 'afr',
 'nld',
 'deu',
 'afr',
 'deu',
 'afr',
 'deu',
 'eng',
 'deu',
 'ara',
 'afr',
 'sqi',
 'cym',
 'deu',
 'afr',
 'nld',
 'lat',
 'nld',
 'deu',
 'deu',
 'nld',
 'deu',
 'afr',
 'nld',
 'afr',
 'nld',
 'afr',
 'deu',
 'afr',
 'deu',
 'nld',
 'nld',
 'deu',
 'nld',
 'cym',
 'rus',
 'nno',
 'deu',
 'afr',
 'afr',
 'afr',
 'afr',
 'afr',
 'deu',
 'afr',
 'nld',
 'nno',
 'nld',
 'nld',
 'deu',
 'nld',
 'afr',
 'afr',
 'afr',
 'afr',
 'deu',
 'afr',
 'afr',
 'cym',
 'swe',
 'swe',
 'afr',
 'nld',
 'afr',
 'afr',
 'afr',
 'nld',
 'afr',
 'nld',
 'deu',
 'afr',
 'afr',
 'afr',
 'lav',
 'afr',
 'deu',
 'afr',
 'nld',
 'afr',
 'nld',
 'afr',
 'ell',
 'nld',
 'nld',
 'afr',


In [51]:
from sklearn.metrics import classification_report
from pprint import pprint

# precision_score(preds, true_label, average=None)
class_names = [get_iso_693_3(decode[s]) for s in range(102)]
# print(true_label)
# print(preds)
pprint(classification_report(true_label, preds, labels=list(range(len(class_names))), target_names=class_names, zero_division=0, digits=5))

('              precision    recall  f1-score   support\n'
 '\n'
 '         afr    0.00000   0.00000   0.00000         0\n'
 '         amh    0.00000   0.00000   0.00000         0\n'
 '         ara    0.00000   0.00000   0.00000         0\n'
 '         asm    0.00000   0.00000   0.00000         0\n'
 '         ast    0.00000   0.00000   0.00000         0\n'
 '         aze    0.00000   0.00000   0.00000         0\n'
 '         bel    0.00000   0.00000   0.00000         0\n'
 '         bul    0.00000   0.00000   0.00000         0\n'
 '         ben    0.00000   0.00000   0.00000         0\n'
 '         bos    0.00000   0.00000   0.00000         0\n'
 '         cat    0.00000   0.00000   0.00000         0\n'
 '         ceb    0.00000   0.00000   0.00000         0\n'
 '         ckb    0.00000   0.00000   0.00000         0\n'
 '         cmn    0.00000   0.00000   0.00000         0\n'
 '         ces    0.00000   0.00000   0.00000         0\n'
 '         cym    0.00000   0.00000   0.00000     